# **Trading Orders - Learning and Visualising**

The best way of learning is by doing. To learn the trading order basics, I am going to write this script and watch closely, how stock market works when placing different order types. As I am a visual learner, most of the steps will be plotted on figures. The emphasis is on the order types and their execution, while experimenting python codes and libraries.

## **What is a Trade Order?**

An order is an instruction to buy or sell on a trading venue such as a stock market, bond market, commodity market, financial derivative market or cryptocurrency exchange. These instructions can be simple or complicated, and can be sent to either a broker or directly to a trading venue via direct market access.

There are some standard instructions for such orders. Two of the most basic stock order types are **market orders** and **limit orders**, but there are several other types such as, **time in force**, **conditional orders**, etc.

## **Stock Data**
To fetch stock data, I use [Yahoo Finance](https://finance.yahoo.com) in Python. It makes relatively easy and accesible to retrieve historical stock data.

In this examle, I am going to use Microsoft's (**MSFT**) hourly stock price in the second half of 2024. The reason is that its price movement does not (seemingly) follow any trend or pattern and thus makes it appropriate for learning how basic trading orders work.

In [2]:
import yfinance as yf

# Downloading hourly stock data and formatting the data frame
stock_data = yf.download("MSFT", start="2024-06-01", end="2024-12-31", interval="60m", progress=False)
stock_data.columns = stock_data.columns.droplevel(1)

In [3]:
import plotly.graph_objects as go

# Create a time series line chart for closing prices
fig = go.Figure(data=go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines'))

# Add a range slider
fig.update_layout(xaxis_rangeslider_visible=True)

# Hide non-trading hours and weekends
fig.update_xaxes(rangebreaks=[dict(bounds=[16, 9], pattern="hour"),
                              dict(bounds=["sat", "mon"], pattern="day of week")])

# Update chart layout
fig.update_layout(
    title="<b>MSFT Closing Prices</b><br><span style='font-size:12px; color:gray;'><i>hourly, second half-year of 2024</i></span>",
    xaxis_title="<b>Date<b>",
    yaxis_title="<b>Closing Price (USD)<b>",
    template="plotly_white",
    margin=dict(l=40, r=40, t=40, b=40),
)

# Show the plot
fig.show()

## **Market Order**

A market order is the most basic type of trade. It's an order to buy or sell it immediately at the next price available. Typically, if you buy a stock, you will pay a price at or near the posted ask. If you're going to sell a stock, you will receive a price at or near the posted bid you see on your screen.

One important thing to remember is that the last traded price is not necessarily the one your market order will get. In fast-moving and volatile markets, prices change fast, and your investment strategies need to account for the cost changing a bit from what was last posted on the screen. The price will remain the same only when the bid/ask price is exactly at the last traded price.

*Market orders do not guarantee a price, but they do guarantee the order is filled immediately.*

**Example:** only the purchasing/selling amount and the exact time is given

- Buying 2 shares at 2024-06-11 12:30
- Selling 1 share at 2024-07-23 10:30
- Buying 1 share at 2024-10-23 15:30
- Selling 2 shares at 2024-11-29 11:30

In [5]:
import pandas as pd

# Creating columns for order type
trades = pd.DataFrame([
    {"type": "buy", "date": "2024-06-11 12:30", "amount": 2},
    {"type": "sell", "date": "2024-07-23 10:30", "amount": 1},
    {"type": "buy", "date": "2024-10-23 15:30", "amount": 1},
    {"type": "sell", "date": "2024-11-29 11:30", "amount": 2},
])
trades["date"] = pd.to_datetime(trades["date"])

stock_data['MarketOrderBuyPrice'] = None
stock_data['MarketOrderSellPrice'] = None
stock_data['MarketOrderProfitOrLoss'] = None

# Assign prices for buy/sell trades
stock_data.loc[trades[trades['type'] == 'buy']['date'], 'MarketOrderBuyPrice'] = stock_data.loc[trades[trades['type'] == 'buy']['date'], 'Close']
stock_data.loc[trades[trades['type'] == 'sell']['date'], 'MarketOrderSellPrice'] = stock_data.loc[trades[trades['type'] == 'sell']['date'], 'Close']


# Calculate MarketOrderProfitOrLoss
buy_price = None  # Track the buy price for each sell order

# Iterate through each trade
for index, trade in trades.iterrows():
    if trade['type'] == 'buy':
        buy_price = stock_data.loc[trade['date'], 'Close']  # Update buy price
    elif trade['type'] == 'sell' and buy_price is not None:
        sell_price = stock_data.loc[trade['date'], 'Close']
        profit_or_loss = (sell_price - buy_price) * trade['amount']
        stock_data.loc[trade['date'], 'MarketOrderProfitOrLoss'] = profit_or_loss
        buy_price = None  # Reset the buy price after a sell

In [ ]:
# Create a time series line chart for closing prices
fig = go.Figure(data=go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Price'))


# Add green dots for buy trades
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MarketOrderBuyPrice'], 
                         mode='markers', name='Buy', marker=dict(color='#8ac926', size=7)))

# Add red dots for sell trades
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MarketOrderSellPrice'], 
                         mode='markers', name='Sell', marker=dict(color='#ef233c', size=7)))

# Add a range slider
fig.update_layout(xaxis_rangeslider_visible=True)

# Hide non-trading hours and weekends
fig.update_xaxes(rangebreaks=[dict(bounds=[16, 9], pattern="hour"),
                              dict(bounds=["sat", "mon"], pattern="day of week")])

# Update chart layout
fig.update_layout(
    title="<b>MSFT - Market Order Examples</b><br><span style='font-size:12px; color:gray;'><i>hourly, second half-year of 2024</i></span>",
    xaxis_title="<b>Date<b>",
    yaxis_title="<b>Closing Price (USD)<b>",
    template="plotly_white",
    margin=dict(l=40, r=40, t=40, b=40),
)

# Show the plot
fig.show()

## **Limit Order**

A limit order, sometimes referred to as a pending order, allows investors to buy and sell securities at a certain price in the future. This type of order is used to execute a trade if the price reaches the preset level; the order will not be filled if the price does not reach this level. In effect, a limit order sets the maximum or minimum price at which you are willing to buy or sell.

A *buy limit* order can only be executed at the limit price or lower.
A *sell limit* order is analogous; it can only be executed at the limit price or higher.


**Example:** only the limits are given (for the sake of simplicity, only one for the whole period)

- Buy limit: $415 or lower
- Sell limit: $435 or higher

In [43]:
# Creating columns for order type
trades = pd.DataFrame([
    {"type": "buy", "at": "415"},
    {"type": "sell", "at": "435"},
])

stock_data['LimitOrderBuyPrice'] = None
stock_data['LimitOrderSellPrice'] = None
stock_data['LimitOrderProfitOrLoss'] = None

# Assign prices for buy/sell trades
buy_triggered = False
buy_price = None

for index, row in stock_data.iterrows():
    if not buy_triggered and row['Close'] <= 415:
        # Assign LimitOrderBuyPrice for the first qualifying buy
        stock_data.at[index, 'LimitOrderBuyPrice'] = row['Close']
        buy_triggered = True
        buy_price = row['Close']
    elif buy_triggered and row['Close'] >= 435:
        # Assign LimitOrderSellPrice for the first qualifying sell after a buy
        stock_data.at[index, 'LimitOrderSellPrice'] = row['Close']
        buy_triggered = False
        sell_price = row['Close']
        profit_or_loss = (sell_price - buy_price)
        stock_data.at[index, 'LimitOrderProfitOrLoss'] = profit_or_loss

In [44]:
# Create a time series line chart for closing prices
fig = go.Figure(data=go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Price'))

fig.add_hline(y=435, line_width=0.5, line_dash="dot", line_color="grey")
fig.add_hline(y=415, line_width=0.5, line_dash="dot", line_color="grey")

# Add green dots for buy trades
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['LimitOrderBuyPrice'], 
                         mode='markers', name='Buy', marker=dict(color='#8ac926', size=7)))

# Add red dots for sell trades
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['LimitOrderSellPrice'], 
                         mode='markers', name='Sell', marker=dict(color='#ef233c', size=7)))


# Add a range slider
fig.update_layout(xaxis_rangeslider_visible=True)

# Hide non-trading hours and weekends
fig.update_xaxes(rangebreaks=[dict(bounds=[16, 9], pattern="hour"),
                              dict(bounds=["sat", "mon"], pattern="day of week")])

# Update chart layout
fig.update_layout(
    title="<b>MSFT - Limit Order Examples</b><br><span style='font-size:12px; color:gray;'><i>hourly, second half-year of 2024</i></span>",
    xaxis_title="<b>Date<b>",
    yaxis_title="<b>Closing Price (USD)<b>",
    template="plotly_white",
    margin=dict(l=40, r=40, t=40, b=40),
)

# Show the plot
fig.show()